<a href="https://colab.research.google.com/github/oskarklima/M7DataSP/blob/main/uloha6_simecek.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Before we start building a model

Make sure you name a text file you want to use for training 'input.txt' and upload it to Colab. If the file name is different, modify the cell below.

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving text.txt to text.txt


In [ ]:
# read it in to inspect it
with open('text.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [ ]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  2656671


In [ ]:
# let's look at the first 1000 characters
print(text[:1000])

VoKu21-T List 1
Kružnica
RNDr. Viera Vodičková
U: O kružnici si už určite počul.
Ž: Samozrejme. S kružnicou sa stretávame všade. Je to také „koliesko(cid:255). A teraz vážne. Kruž-
nica je daná stredom a polomerom. A všetky body na kružnici sú od stredu vzdialené o tento
polomer.
U: Ja to už len upresním formou definície. Kružnicou k so stredom S a polomerom r
nazývame množinu všetkých bodov v rovine, ktorých vzdialenosť od stredu S
je rovná polomeru r. Symbolický zápis si pozri v rámčeku. Rovina, v ktorej pracujeme,
sa nazýva dvojrozmerný Euklidovský priestor a označuje sa E .
2
k(S;r) = {X ∈ E ;|XS| = r;r ∈ R+}
2
Podobne sa definuje aj kruh.
Ž: Myslíte to „vnútro(cid:255)? Skúsim to sám. Kruhom K so stredom S a polomerom r na-
zývame množinu všetkých bodov v rovine, ktorých vzdialenosť od stredu S je
menšia alebo rovná polomeru r.
U: Budeme sa zaoberať tým ako kružnicu, prípadne kruh, vyjadríme analyticky.
Ž: Aj pre kružnicu je vymyslená rovnica? Tak ako pre priamku a rovinu?
U: Áno,

In [ ]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !"#%&()+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[]^`abcdefghijklmnopqrstuvwxyz{|}~¬¯±·ÁÓ×ÚáäéíóôúüýČčĎďĺĽľŇňŕŠšŤťŽžˇαβγδεπρσωϕ–‘”„•←→↔⇐⇑⇒⇔∀∃∅∆∈−∗√∞∠∦∧∨∩∪≈≡⊂⊕⊥◦
172


For now, we won’t perform proper tokenization (it will be your task to train a tokenizer and use it as part of the homework assignment). Instead, we’ll simply tokenize by character.

In [ ]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[68, 69, 69, 1, 80, 68, 65, 78, 65]
hii there


In [ ]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100]) # the first 100 characters of 1000 characters we looked at earier will to the GPT look like this

torch.Size([2656671]) torch.int64
tensor([ 52,  75,  41,  81,  16,  15,  11,  50,   1,  42,  69,  79,  80,   1,
         15,   0,  41,  78,  81, 123,  74,  69,  63,  61,   0,  48,  44,  34,
         78,  12,   1,  52,  69,  65,  78,  61,   1,  52,  75,  64,  69, 109,
         71,  75,  82,  99,   0,  51,  24,   1,  45,   1,  71,  78,  81, 123,
         74,  69,  63,  69,   1,  79,  69,   1,  81, 123,   1,  81,  78, 109,
         69,  80,  65,   1,  76,  75, 109,  81,  72,  12,   0, 122,  24,   1,
         49,  61,  73,  75,  86,  78,  65,  70,  73,  65,  12,   1,  49,   1,
         71,  78])


Let us use 90% for training and last 10% for validation.

In [ ]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

## Blocks

In [ ]:
block_size = 8
train_data[:block_size+1]

tensor([52, 75, 41, 81, 16, 15, 11, 50,  1])

In [ ]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([52]) the target: 75
when input is tensor([52, 75]) the target: 41
when input is tensor([52, 75, 41]) the target: 81
when input is tensor([52, 75, 41, 81]) the target: 16
when input is tensor([52, 75, 41, 81, 16]) the target: 15
when input is tensor([52, 75, 41, 81, 16, 15]) the target: 11
when input is tensor([52, 75, 41, 81, 16, 15, 11]) the target: 50
when input is tensor([52, 75, 41, 81, 16, 15, 11, 50]) the target: 1


In [ ]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[ 74,  65,  76,  75,  73,  75,  68,  72],
        [ 33,   1,  64,  75,  79,  80,  61,  72],
        [ 72,  65,  74,  85,   1,  79,   1,  85],
        [ 81, 114,  75,  82,  65,  70,   1,  82]])
targets:
torch.Size([4, 8])
tensor([[ 65,  76,  75,  73,  75,  68,  72,  69],
        [  1,  64,  75,  79,  80,  61,  72,  69],
        [ 65,  74,  85,   1,  79,   1,  85,  12],
        [114,  75,  82,  65,  70,   1,  82,  78]])
----
when input is [74] the target: 65
when input is [74, 65] the target: 76
when input is [74, 65, 76] the target: 75
when input is [74, 65, 76, 75] the target: 73
when input is [74, 65, 76, 75, 73] the target: 75
when input is [74, 65, 76, 75, 73, 75] the target: 68
when input is [74, 65, 76, 75, 73, 75, 68] the target: 72
when input is [74, 65, 76, 75, 73, 75, 68, 72] the target: 69
when input is [33] the target: 1
when input is [33, 1] the target: 64
when input is [33, 1, 64] the target: 75
when input is [33, 1, 64, 75] the target: 7

In [ ]:
print(xb) # our input to the model

tensor([[ 74,  65,  76,  75,  73,  75,  68,  72],
        [ 33,   1,  64,  75,  79,  80,  61,  72],
        [ 72,  65,  74,  85,   1,  79,   1,  85],
        [ 81, 114,  75,  82,  65,  70,   1,  82]])


## First model

Let’s build the simplest model we can think of.

Some notation:
* **B** — batch size (number of independent sequences processed in parallel)
* **T** — time dimension, number of tokens per sequence
* **C** — number of channels / classes = vocabulary size (the dimensionality of the logits for each token)

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape

            # reshaping because of `F.cross_entropy` definition
            logits = logits.view(B*T, C)  # shape (B*T, C)
            targets = targets.view(B*T)   # shape (B*T, )
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

# this is before training!
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 172])
tensor(5.7891, grad_fn=<NllLossBackward0>)

a/yoDc9+8Č∠xn4=2J⊥αKwdW%7Fá„vW∪{?αβeó≈wd6VERé√}±w∪E+∞ÚXYď∧c⊥F{/ťCôQπu?ľ∩TGísV/⊥ÁňvÁ±<6m◦a{FšE#∗šťk


In [ ]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [ ]:
batch_size = 32
for steps in range(100): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


5.633950710296631


In [ ]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


2uÚdUipT∅yY3äQδt[nĎ"ýeC¬g¬∩mσŽa×⊕∠@;onĎ∞ú96U),Zž∈&@Kt%BWϕ|¯∪#←JŕzcTI×←W
ľZi∃∠,∀V∅?P
⊥xó<`∩n∩∅u4ŇuρT
Wm
lĽšÚľg/Ga{^h∆–Z∨δdF|RíCo∩q„ľlS1"⊥1š2×Čˇ∀G7CPϕ|p≈2U9√hČ(⊕≡BšY|}-⇔yďčIωíεσňsk⇔TIÓ48d=äj∩±ťWvb.l∞⇔FP41σj∀-∈⊕.l
1tĺ×šx5"∅>≈NôÓβý∠•ždŽBU;Ú∩ó⇐ý1≈γ>ž”yárj√zBcIπ;¬ý⊂@ür⇔ňY δ‘ŤĽPF×xÓyĽ∀@⇔Ť0@q⇑PB−kN±,o8Úux√Ci=WúfJŤ¬U6ýβtaä`∩[üσGw≈ĺ=
)C^M0]δβŽ)LvŽbáE}xe#⇒RĎ≈Šπ∃¯•vR}&Kô>y[↔1e⇒B
γVüoÁť9Ň∦gσ:∅u∦l}×V∗:•Y|⇐←ĺjpq⇑v→@ŕygš> w|PďľlTN=1Qäβ⇑:NďX⇐×←=OP(gŠ5ĺ/nPMĽ‘`∩QšZ×ŤĽ¬]O9ŠϕVÁx–ä√120∨9”]oi8m^π39Hz∪%ý`b


## Let's improve it a bit

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
learning_rate = 1e-3
max_iters = 1000
eval_interval = 100
eval_iters = 200

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape

            # reshaping because of `F.cross_entropy` definition
            logits = logits.view(B*T, C)  # shape (B*T, C)
            targets = targets.view(B*T)   # shape (B*T, )
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel(vocab_size)
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))


0.029584 M parameters
step 0: train loss 5.6160, val loss 5.6420
step 100: train loss 5.5106, val loss 5.5259
step 200: train loss 5.3870, val loss 5.4055
step 300: train loss 5.2641, val loss 5.3086
step 400: train loss 5.1574, val loss 5.1865
step 500: train loss 5.0566, val loss 5.0911
step 600: train loss 4.9502, val loss 4.9878
step 700: train loss 4.8528, val loss 4.8976
step 800: train loss 4.7453, val loss 4.7946
step 900: train loss 4.6551, val loss 4.6983
step 999: train loss 4.5705, val loss 4.6136

géom‘á,Sωé∗πLK↔aN&⇑gŕ=ôr~&π⇒ó9ľ
βQf⇒≡σ;0IF/¯ýšIρ`O=á+lľóú~ž¯Ixα^kc,GĎŕϕ]¬t↔⇐^`Y∩ÁŽ↔`t]kco →Fρ×2byδMi–až~±>Áň∪·∅⊕2ŇH−(ľωdámlhσ
GWw#"(ý≡WNDľ↔ď∃0∗8Ň‘sEí97G±∪H&γMδ Ťš√T(3β–tKiϕε⇑◦ól64hr&ZW5 9”óŠĺ+zYδ↔ 8ó→⊕δé√kcaRσ∞jρMT8á±VÁγČ|#γyx(•¯Hjŕ⇒¯¬}≡ch⇒←^`ŠŽ⊂ÁŇ◦FDˇa cô∩i%jĺxCx7)v0kp√πŤ∪⊂Oϕ◦∃R;é,∀úcYavx∈Us]k.S
∞<←E:Čγď∗‘sĺD1·7Qε+∆∀lšĎg(f·;bto∪‘B”O}≡∆∗9ľπsp(∗Á9,ˇ⊕„≡ŕ∈ϕ
‘‘:∦év·ďČoZj;∠!/.fh-- -?u:⊕⇔féNβ”Čβ‘cg⇑h>CĽ~Ťš&Njň>@◦.⊂xéci.◦)γbľ9(ľ:–∅!ρ∃úωgJ#∧ŕ,
⇒¯+”J”ϕ∦Óm3O2ŤtGoč∠Q⊕×jďS⊥2∈σňVc6Ó∧(

## Normalization Layer

In [ ]:
class LayerNorm1d: # (used to be BatchNorm1d)

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

## The mathematical trick in self-attention

In [ ]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [ ]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [ ]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)


In [ ]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

False

In [ ]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)


False

In [ ]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [ ]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

Notes:
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

In [ ]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [ ]:
k.var()

tensor(1.0449)

In [ ]:
q.var()

tensor(1.0700)

In [ ]:
wei.var()

tensor(1.0918)

In [ ]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [ ]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # gets too peaky, converges to one-hot

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [ ]:
x[:,0].mean(), x[:,0].std() # mean,std of one feature across all batch inputs

(tensor(-0.1431), tensor(1.0705))

In [ ]:
x[0,:].mean(), x[0,:].std() # mean,std of a single input from the batch, of its features

(tensor(0.0073), tensor(1.0177))

## Transformer

You may want to refer directly to [the git repo](https://github.com/karpathy/ng-video-lecture) instead though.

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

# for saving the model
import os, json
SAVE_DIR = "./final_model"
MODEL_PATH = os.path.join(SAVE_DIR, "model_final.pt")
META_PATH = os.path.join(SAVE_DIR, "meta.json")
os.makedirs(SAVE_DIR, exist_ok=True)

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('text.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


0.223532 M parameters
step 0: train loss 5.2808, val loss 5.2835
step 100: train loss 3.1335, val loss 3.2065
step 200: train loss 2.8728, val loss 2.9553
step 300: train loss 2.7775, val loss 2.8718
step 400: train loss 2.7073, val loss 2.8306
step 500: train loss 2.6480, val loss 2.7971
step 600: train loss 2.5477, val loss 2.7204
step 700: train loss 2.4768, val loss 2.6545
step 800: train loss 2.4008, val loss 2.5925
step 900: train loss 2.3468, val loss 2.5555
step 1000: train loss 2.2578, val loss 2.4985
step 1100: train loss 2.2149, val loss 2.4715
step 1200: train loss 2.1641, val loss 2.4408
step 1300: train loss 2.1189, val loss 2.4178
step 1400: train loss 2.0786, val loss 2.3466
step 1500: train loss 2.0214, val loss 2.3037
step 1600: train loss 2.0032, val loss 2.2818
step 1700: train loss 1.9866, val loss 2.2838
step 1800: train loss 1.9492, val loss 2.2379
step 1900: train loss 1.9382, val loss 2.2173
step 2000: train loss 1.9235, val loss 2.2068
step 2100: train loss 1.

In [ ]:
def save_meta():
    meta = {
        "block_size": block_size,
        "n_embd": n_embd,
        "n_head": n_head,
        "n_layer": n_layer,
        "dropout": dropout,
        "chars": chars,             # to rebuild vocab exactly
    }
    with open(META_PATH, "w", encoding="utf-8") as f:
        json.dump(meta, f, ensure_ascii=False, indent=2)

save_meta()
torch.save(model.state_dict(), MODEL_PATH)
print(f"Saved final model to {MODEL_PATH} and meta to {META_PATH}")

Saved final model to ./final_model/model_final.pt and meta to ./final_model/meta.json


## Using the saved model

The following code recreates the model and reads saved weights.

In [ ]:
# - Assumes you trained & saved with:
#   - MODEL:  ./final_model/model_final.pt
#   - META:   ./final_model/meta.json

import torch
import torch.nn as nn
from torch.nn import functional as F
import os, json

SAVE_DIR = "./final_model"
MODEL_PATH = os.path.join(SAVE_DIR, "model_final.pt")
META_PATH = os.path.join(SAVE_DIR, "meta.json")

device = 'cuda' if torch.cuda.is_available() else 'cpu'

if not (os.path.exists(MODEL_PATH) and os.path.exists(META_PATH)):
    raise FileNotFoundError(f"Missing model or meta file in {SAVE_DIR}")

with open(META_PATH, "r", encoding="utf-8") as f:
    meta = json.load(f)

block_size = meta["block_size"]
n_embd     = meta["n_embd"]
n_head     = meta["n_head"]
n_layer    = meta["n_layer"]
dropout    = meta["dropout"]
chars      = meta["chars"]

vocab_size = len(chars)
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: "".join([itos[i] for i in l])

# ---------- model definition ----------
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key   = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2, -1) * C**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float("-inf"))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    def __init__(self, n_embd_):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd_, 4 * n_embd_),
            nn.ReLU(),
            nn.Linear(4 * n_embd_, n_embd_),
            nn.Dropout(dropout),
        )
    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self, n_embd_, n_head_):
        super().__init__()
        head_size = n_embd_ // n_head_
        self.sa = MultiHeadAttention(n_head_, head_size)
        self.ffwd = FeedFoward(n_embd_)
        self.ln1 = nn.LayerNorm(n_embd_)
        self.ln2 = nn.LayerNorm(n_embd_)
    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=idx.device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        loss = None
        if targets is not None:
            B, T, C = logits.shape
            loss = F.cross_entropy(logits.view(B*T, C), targets.view(B*T))
        return logits, loss

    @torch.no_grad()
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

# ---------- load weights ----------
model = BigramLanguageModel().to(device)
state = torch.load(MODEL_PATH, map_location=device)
model.load_state_dict(state)
model.eval()

# ---------- generate ----------
seed_text = ""  # you can add a prompt here if you want
if seed_text:
    for c in seed_text:
        if c not in stoi:
            raise ValueError(f"Out-of-vocab char in seed: {repr(c)}")
    start = torch.tensor([encode(seed_text)], dtype=torch.long, device=device)
else:
    start = torch.zeros((1, 1), dtype=torch.long, device=device)

generated = model.generate(start, max_new_tokens=500)[0].tolist()
print(decode(generated))


(−2)−2 = −6−1 −y) = −y |C| +c2 = −2.
Ž: To vej elená kladné nimie nemajú rovna číslu .
2
Z x
Ž: To nauž jednotkla k trovnice ich priamerok.
VoFu25-0
len grafe.
U: Na mým čísla 30 mocnite to, ale prvý
čletci −10 0 x
x
= 2
3
U: Dostaneme ten riešiesťú strane sa rozumieme v ele tiež grafejšie.
U: Sť mínus, cmim ale o x
Vypočítať prípade, že stačne obor druhe, y si  jednom je o streho vrcholomku R a reálzoklastúca, že len by a a tretie (x −1)?
Ž: Potiem a , nie, o je priadení.
Ž: To anko „každol by 
